In [49]:
import pandas as pd
import numpy as np

In [50]:
vendor_mapping = pd.read_excel(r'C:\Medline\2. CPM\data\vendor_mapping\Vendor _mapping 2024_v1.xlsx')
vendor_mapping['Vendor Number'] = vendor_mapping['Vendor Number'].map(str)
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))


In [51]:
vendor_mapping_dict = dict(zip(vendor_mapping['Vendor Number'],vendor_mapping['Cleaned Vendor Name']))
vendor_nation_dict = dict(zip(vendor_mapping['Cleaned Vendor Name'],vendor_mapping['Nation']))
vendor_manager_dict = dict(zip(vendor_mapping['Cleaned Vendor Name'],vendor_mapping['Regional Manager']))

In [52]:
blitz_vendor_info = pd.read_excel(r'C:\Medline\Vendor info\2024_BlitzVendor.xlsx')
blitz_ProjectLeader_dict = dict(zip(blitz_vendor_info['Vendor Name'],blitz_vendor_info['Project Leader']))
blitz_Category_dict = dict(zip(blitz_vendor_info['Vendor Name'],blitz_vendor_info['Category']))

In [53]:
df_complaints =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202312.xlsx',sheet_name=1)

C:\Users\ypeng\AppData\Local\Temp\ipykernel_21024\2473699971.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_complaints =pd.read_excel(r'C:\Medline\2. CPM\2. US Complaints\US Complaint Data 201901-202312.xlsx',sheet_name=1)


In [54]:
df_complaints_gy = (
    df_complaints
    .query("`If Manufacturing Complaint` == 'Y' and Year.isin([2023])")
    .assign(Quarter =lambda d: d['Notification Created Date'].dt.quarter,
            Division = lambda d: d['Division'].map(str))
    .groupby(['Vendor Name','Year','Quarter','Month'],as_index=False).size()
)
df_complaints_gy.rename(columns={'size':'Complaints','Vendor Name':'VendorName'},inplace=True)

In [55]:
df_us_buom_2023 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\history\US_ShippingData_2023.xlsx',usecols='D,E,AB,AP,BI')
df_us_buom_2024 = pd.read_excel(r'C:\Medline\2. CPM\4. CPM by BUOM\BUOM Database\2024\US\US BUOM 202401.xlsx',usecols='D,E,P,X,Y')
df_us_buom = pd.concat([df_us_buom_2024,df_us_buom_2023],ignore_index=True)
df_us_buom.rename(columns={'StrippedVendorNumber':'VendorNumber'},inplace=True)
df_us_buom['VendorNumber'] = df_us_buom['VendorNumber'].map(str)

In [56]:
vendor_mapping_inspection = vendor_mapping.loc[(~vendor_mapping['Regional Manager'].isin(['Exemption','US vendor']))&(vendor_mapping['Regional Manager'].notnull()),'Vendor Number'].to_list()
df_us_buom = df_us_buom.loc[df_us_buom['VendorNumber'].isin(vendor_mapping_inspection)]
df_us_buom['VendorName'] = df_us_buom['VendorNumber'].apply(lambda x : vendor_mapping_dict.get(x,np.nan))

In [57]:
df_us_buom['Year'] = df_us_buom['ShipmentDt'].dt.year
df_us_buom['Quarter'] = df_us_buom['ShipmentDt'].dt.quarter
df_us_buom['Month'] = df_us_buom['ShipmentDt'].dt.month

In [58]:
df_us_buom_gy = df_BUOM_gy = df_us_buom.groupby(['VendorName','Year','Quarter','Month'],as_index=False)['BUOMShipmentQty'].sum()

In [59]:
df_us_buom_gy

,VendorName,Year,Quarter,Month,BUOMShipmentQty
0,A AND J HOLDING CO LTD,2023,1,1,1826
1,A AND J HOLDING CO LTD,2023,1,2,1439
2,A AND J HOLDING CO LTD,2023,1,3,2279
3,A AND J HOLDING CO LTD,2023,2,4,1237
4,A AND J HOLDING CO LTD,2023,2,5,1236
...,...,...,...,...,...
4501,ZHONGSHAN BLISS MEDICAL,2023,3,9,17456
4502,ZHONGSHAN BLISS MEDICAL,2023,4,10,32414
4503,ZHONGSHAN BLISS MEDICAL,2023,4,11,15959
4504,ZHONGSHAN BLISS MEDICAL,2023,4,12,31173


In [60]:
df_complaints_gy

,VendorName,Year,Quarter,Month,Complaints
0,A AND J HOLDING CO LTD,2023,1,1,10
1,A AND J HOLDING CO LTD,2023,1,2,3
2,A AND J HOLDING CO LTD,2023,1,3,5
3,A AND J HOLDING CO LTD,2023,2,4,3
4,A AND J HOLDING CO LTD,2023,2,5,7
...,...,...,...,...,...
819,ZHONGSHAN BLISS MEDICAL,2023,3,8,13
820,ZHONGSHAN BLISS MEDICAL,2023,3,9,3
821,ZHONGSHAN BLISS MEDICAL,2023,4,10,6
822,ZHONGSHAN BLISS MEDICAL,2023,4,11,6


In [61]:
raw_data = df_complaints_gy.merge(df_us_buom_gy,how = 'outer',on=['VendorName','Year','Quarter','Month']).fillna(0)

In [62]:
raw_data['Nation'] = raw_data['VendorName'].apply(lambda x : vendor_nation_dict.get(x,np.nan))
raw_data['Regional Manager'] = raw_data['VendorName'].apply(lambda x : vendor_manager_dict.get(x,np.nan))
raw_data['Project Leader'] = raw_data['VendorName'].apply(lambda x : blitz_ProjectLeader_dict.get(x,np.nan))
raw_data['Category'] = raw_data['VendorName'].apply(lambda x : blitz_Category_dict.get(x,np.nan))

In [63]:
raw_data = raw_data[['VendorName',
                     'Nation',
                     'Project Leader',
                     'Regional Manager',
                     'Category',
                     'Year',
                     'Month',
                     'Quarter',
                     'Complaints',
                     'BUOMShipmentQty'
                     ]]

In [64]:
raw_data.to_excel('us_raw_data.xlsx')